In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Set the paths to the dataset directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'
val_dir = 'dataset/val'

In [5]:
# Define the image dimensions and batch size
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

In [7]:
# Create ImageDataGenerators for training, test, and validation sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [9]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=True
)


Found 41185 images belonging to 5 classes.


In [10]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=False  # No need to shuffle test data
)

Found 11772 images belonging to 5 classes.


In [13]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=False  # No need to shuffle validation data
)

Found 5882 images belonging to 5 classes.


In [15]:
# Define the input shape
input_shape = train_generator.image_shape

In [17]:
# Define the input tensor
input_img = Input(shape=input_shape)


In [19]:
# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(x)


In [21]:
# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


In [23]:
# Create the Autoencoder model
autoencoder = Model(input_img, decoded)

In [25]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [27]:
# Train the Autoencoder
history = autoencoder.fit(
    train_generator,
    epochs=30,
    batch_size=BATCH_SIZE,
    validation_data=val_generator
)

Epoch 1/30


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1288/1288 ━━━━━━━━━━━━━━━━━━━━ 459s 355ms/step - loss: 0.0050 - val_loss: 7.2625e-04
Epoch 2/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 457s 355ms/step - loss: 7.7452e-04 - val_loss: 5.8110e-04
Epoch 3/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 457s 354ms/step - loss: 6.4300e-04 - val_loss: 5.2309e-04
Epoch 4/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 462s 359ms/step - loss: 5.7040e-04 - val_loss: 5.0610e-04
Epoch 5/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 453s 351ms/step - loss: 5.2303e-04 - val_loss: 4.8312e-04
Epoch 6/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 459s 356ms/step - loss: 4.8623e-04 - val_loss: 4.3566e-04
Epoch 7/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 459s 356ms/step - loss: 4.4560e-04 - val_loss: 4.0299e-04
Epoch 8/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 461s 358ms/step - loss: 4.2202e-04 - val_loss: 3.8304e-04
Epoch 9/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 457s 355ms/step - loss: 3.9607e-04 - val_loss: 3.9407e-04
Epoch 10/30
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 458s 355ms/step - loss: 3.7825e-04 - val_loss: 3.2904e-04
Epoch 11/30
12

In [29]:
# Evaluate the Autoencoder on the test set
test_loss = autoencoder.evaluate(test_generator)


368/368 ━━━━━━━━━━━━━━━━━━━━ 51s 139ms/step - loss: 2.3618e-04


In [33]:
# Save the trained Autoencoder model
autoencoder.save('../autoencoder_model/autoencoder2.keras')
